## Potential Pipeline Process

Process that:

1. fetches building footprints from OSM (OSMnx)

2. reads your 15-minute demand tables from Supabase and computes annual energy need

3. downloads GeoTIFFs from Supabase to compute a simple shade index per building

4. estimates roof orientation, tilt, area, and per-building annual solar energy potential

5. computes a composite solar_suitability score for each building

6. selects a set of buildings (residential vs commercial) to meet the user’s chosen percentage of city power from solar while trying to respect the chosen commercial/building mix

7. visualizes everything on a PyDeck map (colored polygons by solar_score and highlighted chosen buildings)

8. writes results back to Supabase as GeoJSON in a building_suitability table

Modularized so we can swap in better irradiance, panel efficiency, or LIDAR later.

In [ ]:
# Imports
#pip install streamlit supabase osmnx geopandas rasterio shapely pyproj pydeck
# all possibles so far, may need to add more as necessary
import streamlit as st
from supabase import create_client, Client
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import tempfile
import rasterio
from rasterio.mask import mask
import osmnx as ox
from shapely.geometry import Polygon, mapping
import pydeck as pdk
import json
from datetime import datetime, timedelta

In [ ]:
st.set_page_config(layout="wide", page_title="Solar Suitability Planner")

# This is how I set up the secrets on my machine, not the same as Postgres!
url = st.secrets["SUPABASE_URL"]
key = st.secrets["SUPABASE_KEY"]
supabase: Client = create_client(url, key)

# Default average daily insolation in kWh/m2/day (tweakable)
DEFAULT_INSOLATION = {
    "Ann Arbor": 4.0,   # ~kWh/m2/day 
    "Tucson": 6.0,
}
PANEL_EFFICIENCY = 0.18   # guessing, can replace from PV avgs
PERFORMANCE_RATIO = 0.75  # guessing, can replace from PV avgs

In [ ]:
# WORKING
def load_demand_table(city: str) -> pd.DataFrame:
    """
    Load city demand table from Supabase for selected city
    Expected numeric column: MW and datetime column named 'date_time'.
    """
    table_name = "Ann_Arbor_demand" if city == "Ann Arbor" else "TEPC_demand"
    res = supabase.table(table_name).select("*").execute()
    data = res.data
    df = pd.DataFrame(data)
    return df

In [ ]:
# WORKING
# Takes a while to run: 3m 13s for Tucson!
def fetch_buildings_osm(place_name: str) -> tuple:
    """
    Use OSMnx to fetch building footprints for the given place name.
    Returns GeoDataFrame with area_m2 computed per building and total area.
    """
    tags = {"building": True}
    gdf = ox.features.features_from_place(place_name, tags=tags)
    
    # Filter to only Polygons and MultiPolygons
    gdf = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]
    
    # Residential/Commercial Classification
    gdf["is_commercial"] = gdf.apply(
        lambda row: (
            (pd.notna(row.get("building")) and str(row.get("building")).lower() in
             ["commercial", "retail", "industrial", "office", "warehouse"])
        ), axis=1
    )

    # Calculate the area (meters)
    gdf_proj = ox.projection.project_gdf(gdf)
    gdf['area_m2'] = gdf_proj.area

    # Total area (sq m) covered by building footprints - this will go to City Specs
    total_area = gdf['area_m2'].sum()

    return gdf, print(total_area)   # remove print() for production

In [ ]:
# For using TIFs to obtain shading, tilt, etc
# Need to set up paths correctly and make sure all images in buckets

def download_geotiff_from_supabase(bucket: str, path: str) -> rasterio.io.DatasetReader:
    """
    Download a file from Supabase storage bucket to a temp file and open with rasterio.
    """

In [ ]:
def shade_from_geotiff(raster: rasterio.io.DatasetReader, polygon: Polygon):
    """
    Compute a simple shade score (0 bright / 1 shaded) by masking the raster to the polygon and computing mean brightness.
    Assumes first band is usable (grayscale or visible).
    """

In [ ]:
def compute_city_annual_kwh(df: pd.DataFrame):
    """
    Convert the 15-minute MW measurements into annual kWh for each city.
    """

In [ ]:
# May be able to calculate using saved Azimuth values and MATH

def orientation_match_score(roof_angle_deg: float, ideal_sun_azimuth_deg: float):
    """
    Compute orientation match score 0-1 where 1 = perfect face to sun
    """

In [ ]:
def compute_suitability_scores(gdf: gpd.GeoDataFrame, irradiance_factor: float, ideal_sun_azimuth: float):
    """
    Calculate final solar_score from area, and if possible: orientation, tilt, shade, and irradiance.
    Compute intermediate scores in [0,1] and combine with weights.
    """

In [ ]:
def estimate_building_annual_potential_kwh(gdf: gpd.GeoDataFrame, insolation_kwh_m2_day: float,
                                           panel_efficiency=PANEL_EFFICIENCY, perf_ratio=PERFORMANCE_RATIO):
    """
    Estimate the annual kWh each building can produce.
    """

In [ ]:
def select_buildings_to_meet_target(gdf: gpd.GeoDataFrame, required_kwh: float, commercial_pct: float):
    """
    Greedy selection that alternates choosing commercial/residential in a ratio
    that tries to match the requested commercial_pct while picking highest solar_score first.
    Returns GeoDataFrame of selected buildings and remaining totals.
    """

In [ ]:
# Results summary

In [ ]:
# Save geometry results to Supabase

Notes:

-might use LiDAR (DEM) for true solar path shading

-currently using default irradiance values (Ann Arbor ~4, Tucson ~6 kWh/m²/day), but maybe can be replaced with city-specific measured irradiance (NREL) for better accuracy

-add an endpoint to export selected parcels as GeoJSON/CSV

-store results back in Supabase (then the app only reads results)